In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from harvesting_tools.harvest_paths import SetupPaths

In [2]:
path_to_harvest = "/xdisk/gbesla/katiechambe/harvest"
paths = SetupPaths(path_to_harvest)

# Check for common ids

In [3]:
# check if there are any common ids between snapshot 40 and 41
orbit_dict = {}
for snap in [40,41,42]:
    f = h5py.File(f"{paths.path_orbits}/highmass_major_{snap}.hdf5",'r')
    orbit_dict[str(snap)] = {}
    for key, val in f.items():
        orbit_dict[str(snap)][key] = np.array(val)
    f.close()

f = h5py.File(f"{paths.path_misc}snapshot_data.hdf5",'r')
snap_dict = {}
for key, val in f.items():
    snap_dict[key] = np.array(val)
f.close()

In [4]:
xx = orbit_dict['40']['PairKey']
yy = orbit_dict['41']['PairKey']
zz = orbit_dict['42']['PairKey']

In [5]:
print(len(xx),len(yy),len(zz))

212 237 241


In [9]:
alpha = np.concatenate((xx,yy,zz))
print("Total num of pairs in snaps 40-42:",len(alpha))
print("Number of unique pairs in snaps 40-42:",len(np.unique(alpha)))

Total num of pairs in snaps 40-42: 690
Number of unique pairs in snaps 40-42: 369


# Compile all unique orbits into new data files

In [16]:
def get_unique(size="high", type="major"):
    # initialize a dict with all the keys from any one of the orbit files
    unique_frame = {}
    for i in orbit_dict['40'].keys():
        unique_frame[i] = []
        
    # go through each snapshot and add orbits with unique keys:
    for snapnum in range(100):    
        if snapnum == 48:
            continue
        
        orbit = {}
        f = h5py.File(f"{paths.path_orbits}/{size}mass_{type}_{snapnum}.hdf5",'r')
        for key, val in f.items():
            orbit[key] = np.array(val)
        f.close()
    
        for hua in range(len(orbit['PairKey'])):
            pk = orbit['PairKey'][hua]
            if pk in unique_frame['PairKey']:
                continue
            else:
                for key in unique_frame.keys():
                    if key not in ["Redshift","Scale","Snapshot"]:
                        unique_frame[key].append(orbit[key][hua])
                    elif key=="Snapshot":
                        unique_frame[key].append(snapnum)

    return unique_frame

                


In [17]:
info_dict = {"Redshift":"Redshift of snapshot",
              "Scale":"Scale of snapshot",
              "Snapshot":"Snapshot number",
              "GroupNum":"FoF group number at snapshot",
              "SubfindID1":"Subhalo ID of primary at selected redshift",
              "SubfindID2":"Subhalo ID of secondary at selected redshift",
              "SubhaloMass1":"Subhalo mass at selected redshift in 1e10*Msun",
              "SubhaloMass2":"Subhalo mass at selected redshift in 1e10*Msun",
              "StellarMass1":"Stellar mass from median AM at selected redshift",
              "StellarMass2":"Stellar mass from median AM at selected redshift",
              "StellarMassRatio":"Stellar mass ratio at selected redshift",
              "MergeFlag":"True if subhalos will merge", 
              "MergeRedshift":"Redshift of 'merger'",
              "MergeSnapshot":"Snapshot at which 'merger' has occured",
              "InfallRedshift":"First redshift where Group is the same",
              "InfallSnapshot":"First snapshot where Group is the same",
              "PairKey":"Unique identifying key for each pair (same between snapshots for same pair)",
              "GroupFlag":"True if subhalos are in the same group",
              "GroupRvir":"Radius of primary group in kpc",
              "Separations":"Physical separation between pair in kpc",
              "SeparationsComoving":"Comoving separation between pair in ckpc/h",
              "SeparationsScaled":"Separation scaled by radius of primary group, dimensionless",
              "RelativeVelocity":"Relative velocity between pair in km/s",
              "SubhaloPos1":"Position of subhalo in ckpc/h",
              "SubhaloPos2":"Position of subhalo in ckpc/h",
              "SubhaloVel1":"Velocity of subhalo in km/s",
              "SubhaloVel2":"Velocity of subhalo in km/s"
            }

In [18]:
for size in ["high","low"]:
    for type in ["major","minor"]:
        unique_data = get_unique(size, type)
        
        f = h5py.File(f"{paths.path_orbits}unique-orbits/{size}mass-{type}.hdf5", 'w')
        header_dict = {"Details":f"Full UNIQUE sample of {size} mass {type} pair orbits",
                "Simulation":"TNG100-1"}
        
        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
        
        for key, val in unique_data.items():
            if key not in ["Redshift","Scale"]:
                val = np.array(val)
                dset = f.create_dataset(f'/{key}', 
                                        shape=val.shape,
                                        dtype=val.dtype)
                dset.attrs[key] = info_dict[key]
                dset[:] = val
        
        f.close()
        print(f"Completed {size} mass {type}")

Completed high mass major
Completed high mass minor
Completed low mass major
Completed low mass minor
